In [ ]:
# Q-Learning



import codey, event, time, rocky, random, ujson, urequests, json
import time, random
import math




states = [0,1,2,3]
actions = [1,2]
Q = [[0 for _ in range(len(actions))] for _ in range(len(states))]





class QLearning:
    """
        Initialize the Q-Learning object with the following parameters:
        - states: The total number of states.
        - actions: The total number of actions.
        - lr: The learning rate for Q-Learning.
        - gamma: The discount factor for future rewards.
    """





    def __init__(self, states, actions, lr=0.05, gamma=0.99):
        self.states = states
        self.actions = actions
        self.lr = lr
        self.gamma = gamma
        # Initialize Q-table with zeros
        self.q_table = [[0 for _ in range(actions)] for _ in range(states)]





    def update(self, state, action, reward, next_state):
        """
        Update the Q-table given a state, action, reward, and the next state.
        """
        future_rewards = max(self.q_table[next_state])
        target = reward + self.gamma * future_rewards
        self.q_table[state][action] = (1 - self.lr) * self.q_table[state][action] + self.lr * target

        return self.q_table




    def get_action(self, state, epsilon):
        """
        Select an action for a given state using epsilon-greedy action selection.
        """
        if random.getrandbits(8) < 255 * epsilon:
            return random.getrandbits(8) % self.actions
        else:
            return self.q_table[state].index(max(self.q_table[state]))





# Define a pseudo-environment
class PseudoEnvironment:
    def __init__(self):
        """
        Initialize the PseudoEnvironment object.
        """
        self.current_state = 0 # Start with state 1
        self.last_transition_time = codey.get_timer()





    def get_color(self):
        if rocky.color_ir_sensor.is_color('black'):
            self.current_state = 0
        if rocky.color_ir_sensor.is_color('green'):
            self.current_state = 1
        if rocky.color_ir_sensor.is_color('white'):
            self.current_state = 2
        if rocky.color_ir_sensor.is_color('blue'):
            self.current_state = 3
        return self.current_state






    def step(self, action):
        reward = 0  # Default reward





        if self.current_state == 0:
            if rocky.color_ir_sensor.is_color('black'):
                rocky.drive(50, 0)
            else:
                rocky.drive(0, 50)



        if self.current_state == 1:
            if action == 1:
                rocky.turn_left_by_degree(90)
                rocky.forward(20, 1)
                while self.current_state == 1:
                    if rocky.color_ir_sensor.is_color('black'):
                        rocky.drive(0, 40)
                    else:
                        rocky.drive(40, 0)
                    if rocky.color_ir_sensor.is_color('white'):
                        break
            else:
                rocky.turn_right_by_degree(90)
                rocky.forward(20, 1)
                while self.current_state == 1:
                    if rocky.color_ir_sensor.is_color('black'):
                        rocky.drive(40, 0)
                    else:
                        rocky.drive(0, 40)
                    if rocky.color_ir_sensor.is_color('white'):
                        break
            current_time = codey.get_timer()
            time_elapsed = current_time - self.last_transition_time
            self.last_transition_time = codey.get_timer()



            reward =  1 - 0.1 * time_elapsed
        elif self.current_state == 2:
            if action == 1:
                rocky.turn_left_by_degree(90)
                rocky.forward(20, 1)
                while self.current_state == 2:
                    if rocky.color_ir_sensor.is_color('black'):
                        rocky.drive(0, 40)
                    else:
                        rocky.drive(40, 0)
                    if rocky.color_ir_sensor.is_color('blue'):
                        break
            else:
                rocky.turn_right_by_degree(90)
                rocky.forward(20, 0.5)
                while self.current_state == 2:
                    if rocky.color_ir_sensor.is_color('black'):
                        rocky.drive(30, 0)
                    else:
                        rocky.drive(0, 30)
                    if rocky.color_ir_sensor.is_color('blue'):
                        break



            current_time = codey.get_timer()
            time_elapsed = current_time - self.last_transition_time
            self.last_transition_time = codey.get_timer()
            reward =  1 - 0.1 * time_elapsed




        elif self.current_state == 3:




            current_time = codey.get_timer()
            codey.display.show(current_time)
            time_elapsed = current_time - self.last_transition_time
            self.last_transition_time = codey.get_timer()
            reward = 100 + (1 - 0.1 * time_elapsed)
            rocky.stop()



        next_state = self.current_state + 1
        if (next_state > 3):
            next_state = 0




        return next_state, reward, time_elapsed, current_time





# Training loop for Q-Learning
@event.button_a_pressed
def on_button_a_pressed():
    global Q

    states = [0, 1, 2, 3]
    actions = [1, 2]


    q_learning = QLearning(states=4, actions=2, lr=0.5, gamma=0.95)
    env = PseudoEnvironment()




    # Initialize lists to store total rewards per episode
    rewards_q_learning = [ ]
    time = []




    # Number of episodes for Q-Learning
    num_episodes = 10



    for i in range(num_episodes):
        codey.display.show('E:'+str(i))
        total_reward = 0
        codey.reset_timer()



        while(1):
            state = env.get_color() # 1
            codey.display.show(state)
            if state != 0:
                action = q_learning.get_action(state, epsilon=0.5)
                next_state, reward, time_elapsed, current_time = env.step(action)
                #codey.display.show("N:"+str(next_state))
                q_learning.update(state, action, reward, next_state)
                total_reward += reward
                rewards_q_learning.append(total_reward)
                time.append(time_elapsed)

                while(env.get_color() != next_state):
                    #codey.display.show(str(env.get_color())+'->'+str(next_state))
                    continue
                if next_state == 0:
                    break

            else:
                while state == env.get_color():
                    if rocky.color_ir_sensor.is_color('black'):
                        rocky.drive(30, 0)
                    else:
                        rocky.drive(0, 30)
                while(state == env.get_color()):
                    continue

        Q = q_learning.update(state, action, reward, next_state)
        c_time = current_time
        print(Q)
        print(rewards_q_learning)
        print(c_time)